In [1]:
import findspark
import itertools

findspark.init()

from pyspark import SparkContext
sc=SparkContext("local","quiz")

21/10/07 20:18:11 WARN Utils: Your hostname, jeongsobin-ui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.9 instead (on interface en0)
21/10/07 20:18:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/10/07 20:18:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
import itertools

def union(x,y):
    if x is None:
        result=y
    elif y is None:
        result=x
    else:
        result=list(set(x) | set(y))
    return result



def flatten(x):
    tmp=[]
    for i in x[0]:
        for j in i:
            tmp.append(j)
    for i in x[1]:
        for j in i:
            tmp.append(j)
    return tmp

In [7]:
data=sc.textFile("facebook_combined.txt").map(lambda x:tuple(x.split(" ")))

In [8]:
data_1_t_g=data.groupByKey().mapValues(list) #아이디를 키로해서 친구들 리스트

inv_data_1_t_g=data.map(lambda x:(x[1],x[0])).groupByKey().mapValues(list) #undirected로 구성되있어서 반대의 경우로도 생성

#생성한 두개의 rdd합쳐서 cogroup
total=data_1_t_g.cogroup(inv_data_1_t_g).mapValues(lambda t:(list(t[0]),list(t[1]))).map(lambda x:(x[0],flatten(x[1])))

In [9]:
overlap=total.map(lambda x:x[1]).flatMap(lambda x:list(itertools.combinations((x),2))).map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y)

In [10]:
overlap.collect()

/opt/homebrew/Cellar/apache-spark/3.1.2/libexec/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling


[(('1', '2'), 1),
 (('1', '3'), 2),
 (('1', '4'), 1),
 (('1', '5'), 2),
 (('1', '6'), 1),
 (('1', '7'), 3),
 (('1', '8'), 1),
 (('1', '9'), 6),
 (('1', '10'), 1),
 (('1', '11'), 1),
 (('1', '12'), 1),
 (('1', '13'), 3),
 (('1', '14'), 1),
 (('1', '15'), 1),
 (('1', '16'), 1),
 (('1', '17'), 1),
 (('1', '18'), 1),
 (('1', '19'), 1),
 (('1', '20'), 1),
 (('1', '21'), 7),
 (('1', '22'), 1),
 (('1', '23'), 1),
 (('1', '24'), 6),
 (('1', '25'), 7),
 (('1', '26'), 7),
 (('1', '27'), 3),
 (('1', '28'), 1),
 (('1', '29'), 1),
 (('1', '30'), 5),
 (('1', '31'), 2),
 (('1', '32'), 1),
 (('1', '33'), 1),
 (('1', '34'), 1),
 (('1', '35'), 1),
 (('1', '36'), 1),
 (('1', '37'), 1),
 (('1', '38'), 2),
 (('1', '39'), 3),
 (('1', '40'), 5),
 (('1', '41'), 1),
 (('1', '42'), 1),
 (('1', '43'), 1),
 (('1', '44'), 1),
 (('1', '45'), 2),
 (('1', '46'), 1),
 (('1', '47'), 1),
 (('1', '48'), 9),
 (('1', '49'), 1),
 (('1', '50'), 2),
 (('1', '51'), 1),
 (('1', '52'), 1),
 (('1', '53'), 9),
 (('1', '54'), 2),
 

In [11]:
#broadcasting생성(친구정보 전체 포함)
dictionary=dict(total.map(lambda x:list(x)).collect())
bbc=sc.broadcast(dictionary)

In [12]:
#각 키를 기준으로 모든 순열조합을 생성해 overlap rdd생성
overlap=total.map(lambda x:x[1]).flatMap(lambda x:list(itertools.combinations((x),2))).map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y)

In [13]:
#total.map(lambda x:x[1]).flatMap(lambda x:list(itertools.combinations((x),2))).map(lambda x:(x,1)).takeSample(False,1000)

In [14]:
overlap.collect()

[(('1', '2'), 1),
 (('1', '3'), 2),
 (('1', '4'), 1),
 (('1', '5'), 2),
 (('1', '6'), 1),
 (('1', '7'), 3),
 (('1', '8'), 1),
 (('1', '9'), 6),
 (('1', '10'), 1),
 (('1', '11'), 1),
 (('1', '12'), 1),
 (('1', '13'), 3),
 (('1', '14'), 1),
 (('1', '15'), 1),
 (('1', '16'), 1),
 (('1', '17'), 1),
 (('1', '18'), 1),
 (('1', '19'), 1),
 (('1', '20'), 1),
 (('1', '21'), 7),
 (('1', '22'), 1),
 (('1', '23'), 1),
 (('1', '24'), 6),
 (('1', '25'), 7),
 (('1', '26'), 7),
 (('1', '27'), 3),
 (('1', '28'), 1),
 (('1', '29'), 1),
 (('1', '30'), 5),
 (('1', '31'), 2),
 (('1', '32'), 1),
 (('1', '33'), 1),
 (('1', '34'), 1),
 (('1', '35'), 1),
 (('1', '36'), 1),
 (('1', '37'), 1),
 (('1', '38'), 2),
 (('1', '39'), 3),
 (('1', '40'), 5),
 (('1', '41'), 1),
 (('1', '42'), 1),
 (('1', '43'), 1),
 (('1', '44'), 1),
 (('1', '45'), 2),
 (('1', '46'), 1),
 (('1', '47'), 1),
 (('1', '48'), 9),
 (('1', '49'), 1),
 (('1', '50'), 2),
 (('1', '51'), 1),
 (('1', '52'), 1),
 (('1', '53'), 9),
 (('1', '54'), 2),
 

In [1]:
#braodcasting이용해서 jaccard coefficient계산을 위해 합집합,교집합이 필요해 각 키에대한 모든 리스트정보 불러온 rdd생성
all_list=overlap.map(lambda x:(x[0],x[1],bbc.value[x[0][0]],bbc.value[x[0][1]]))
all_list.collect()

NameError: name 'overlap' is not defined

In [16]:
#교집합/합집합 이용해 jaccard구현 교집합은 overlap이용
final=all_list.map(lambda x:[x[0],x[1],len(union(x[2],x[3]))])

In [17]:
final.collect()

[[('1', '2'), 1, 26],
 [('1', '3'), 2, 32],
 [('1', '4'), 1, 26],
 [('1', '5'), 2, 28],
 [('1', '6'), 1, 22],
 [('1', '7'), 3, 34],
 [('1', '8'), 1, 24],
 [('1', '9'), 6, 68],
 [('1', '10'), 1, 26],
 [('1', '11'), 1, 17],
 [('1', '12'), 1, 17],
 [('1', '13'), 3, 45],
 [('1', '14'), 1, 31],
 [('1', '15'), 1, 17],
 [('1', '16'), 1, 25],
 [('1', '17'), 1, 29],
 [('1', '18'), 1, 17],
 [('1', '19'), 1, 32],
 [('1', '20'), 1, 31],
 [('1', '21'), 7, 75],
 [('1', '22'), 1, 27],
 [('1', '23'), 1, 33],
 [('1', '24'), 6, 27],
 [('1', '25'), 7, 79],
 [('1', '26'), 7, 78],
 [('1', '27'), 3, 19],
 [('1', '28'), 1, 29],
 [('1', '29'), 1, 29],
 [('1', '30'), 5, 29],
 [('1', '31'), 2, 38],
 [('1', '32'), 1, 22],
 [('1', '33'), 1, 18],
 [('1', '34'), 1, 21],
 [('1', '35'), 1, 18],
 [('1', '36'), 1, 27],
 [('1', '37'), 1, 17],
 [('1', '38'), 2, 24],
 [('1', '39'), 3, 29],
 [('1', '40'), 5, 56],
 [('1', '41'), 1, 40],
 [('1', '42'), 1, 18],
 [('1', '43'), 1, 17],
 [('1', '44'), 1, 22],
 [('1', '45'), 2, 2

In [35]:
#threshold 바꿔가며 결과값 도출
threshold=0.8
upper_th=final.filter(lambda x:x[1]/x[2]>=threshold).map(lambda x:(x[0],x[1]/x[2]))
upper_th.collect()

/opt/homebrew/Cellar/apache-spark/3.1.2/libexec/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling


[(('11', '12'), 1.0),
 (('11', '15'), 1.0),
 (('11', '18'), 1.0),
 (('11', '37'), 1.0),
 (('11', '43'), 1.0),
 (('11', '74'), 1.0),
 (('11', '114'), 1.0),
 (('11', '209'), 1.0),
 (('11', '210'), 1.0),
 (('11', '215'), 1.0),
 (('11', '287'), 1.0),
 (('11', '292'), 1.0),
 (('11', '335'), 1.0),
 (('12', '15'), 1.0),
 (('12', '18'), 1.0),
 (('12', '37'), 1.0),
 (('12', '43'), 1.0),
 (('12', '74'), 1.0),
 (('12', '114'), 1.0),
 (('12', '209'), 1.0),
 (('12', '210'), 1.0),
 (('12', '215'), 1.0),
 (('12', '287'), 1.0),
 (('12', '292'), 1.0),
 (('12', '335'), 1.0),
 (('15', '18'), 1.0),
 (('15', '37'), 1.0),
 (('15', '43'), 1.0),
 (('15', '74'), 1.0),
 (('15', '114'), 1.0),
 (('15', '209'), 1.0),
 (('15', '210'), 1.0),
 (('15', '215'), 1.0),
 (('15', '287'), 1.0),
 (('15', '292'), 1.0),
 (('15', '335'), 1.0),
 (('18', '37'), 1.0),
 (('18', '43'), 1.0),
 (('18', '74'), 1.0),
 (('18', '114'), 1.0),
 (('18', '209'), 1.0),
 (('18', '210'), 1.0),
 (('18', '215'), 1.0),
 (('18', '287'), 1.0),
 (('18

In [11]:
upper_th.collect()

[(('4', '78'), 0.7272727272727273),
 (('8', '91'), 0.7777777777777778),
 (('11', '12'), 1.0),
 (('11', '15'), 1.0),
 (('11', '18'), 1.0),
 (('11', '37'), 1.0),
 (('11', '43'), 1.0),
 (('11', '74'), 1.0),
 (('11', '114'), 1.0),
 (('11', '209'), 1.0),
 (('11', '210'), 1.0),
 (('11', '215'), 1.0),
 (('11', '287'), 1.0),
 (('11', '292'), 1.0),
 (('11', '335'), 1.0),
 (('12', '15'), 1.0),
 (('12', '18'), 1.0),
 (('12', '37'), 1.0),
 (('12', '43'), 1.0),
 (('12', '74'), 1.0),
 (('12', '114'), 1.0),
 (('12', '209'), 1.0),
 (('12', '210'), 1.0),
 (('12', '215'), 1.0),
 (('12', '287'), 1.0),
 (('12', '292'), 1.0),
 (('12', '335'), 1.0),
 (('15', '18'), 1.0),
 (('15', '37'), 1.0),
 (('15', '43'), 1.0),
 (('15', '74'), 1.0),
 (('15', '114'), 1.0),
 (('15', '209'), 1.0),
 (('15', '210'), 1.0),
 (('15', '215'), 1.0),
 (('15', '287'), 1.0),
 (('15', '292'), 1.0),
 (('15', '335'), 1.0),
 (('18', '37'), 1.0),
 (('18', '43'), 1.0),
 (('18', '74'), 1.0),
 (('18', '114'), 1.0),
 (('18', '209'), 1.0),
 ((

In [12]:
#친구 제거
upper_th.map(lambda x:(x[0])).filter(lambda x: x[1] not in bbc.value[x[0]]).collect()

[('11', '12'),
 ('11', '15'),
 ('11', '18'),
 ('11', '37'),
 ('11', '43'),
 ('11', '74'),
 ('11', '114'),
 ('11', '209'),
 ('11', '210'),
 ('11', '215'),
 ('11', '287'),
 ('11', '292'),
 ('11', '335'),
 ('12', '15'),
 ('12', '18'),
 ('12', '37'),
 ('12', '43'),
 ('12', '74'),
 ('12', '114'),
 ('12', '209'),
 ('12', '210'),
 ('12', '215'),
 ('12', '287'),
 ('12', '292'),
 ('12', '335'),
 ('15', '18'),
 ('15', '37'),
 ('15', '43'),
 ('15', '74'),
 ('15', '114'),
 ('15', '209'),
 ('15', '210'),
 ('15', '215'),
 ('15', '287'),
 ('15', '292'),
 ('15', '335'),
 ('18', '37'),
 ('18', '43'),
 ('18', '74'),
 ('18', '114'),
 ('18', '209'),
 ('18', '210'),
 ('18', '215'),
 ('18', '287'),
 ('18', '292'),
 ('18', '335'),
 ('37', '43'),
 ('37', '74'),
 ('37', '114'),
 ('37', '209'),
 ('37', '210'),
 ('37', '215'),
 ('37', '287'),
 ('37', '292'),
 ('37', '335'),
 ('43', '74'),
 ('43', '114'),
 ('43', '209'),
 ('43', '210'),
 ('43', '215'),
 ('43', '287'),
 ('43', '292'),
 ('43', '335'),
 ('74', '114'

In [36]:
# 친구가 한명이여서 1나온경우 제외하면
final=final.filter(lambda x:[x[1],x[2]]!=[1,1])

In [38]:
upper_th=final.filter(lambda x:x[1]/x[2]>=threshold).map(lambda x:(x[0],x[1]/x[2]))
upper_th.map(lambda x:(x[0])).filter(lambda x: x[1] not in bbc.value[x[0]]).collect()

[('90', '145'),
 ('110', '201'),
 ('193', '201'),
 ('201', '245'),
 ('241', '255'),
 ('1232', '1319'),
 ('2704', '2740'),
 ('652', '673'),
 ('595', '599'),
 ('878', '882'),
 ('2972', '2982'),
 ('1992', '2076'),
 ('3481', '3523'),
 ('3854', '3879')]

21/10/07 16:56:17 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 126880 ms exceeds timeout 120000 ms
21/10/07 16:56:17 WARN SparkContext: Killing executors is not supported by current scheduler.
